In [ ]:
%matplotlib widget
%matplotlib inline

In [ ]:
import pandas as pd
from dask import dataframe as dd
import numpy as np

import matplotlib
import matplotlib.cm as cm
import matplotlib.pyplot as plt

#### read databases (parquet files don't include agg points)

In [ ]:
%%time
#READ DATABASE FOR RANDOM ORIENTATION
df_rand = dd.read_parquet("../instance_files/parquet_files/createdb_iceagg_rand*", engine="pyarrow").compute()

In [ ]:
%%time
#READ DATABASE FOR flat ORIENTATION
df_flat = dd.read_parquet("../instance_files/parquet_files/createdb_iceagg_flat*", engine="pyarrow").compute()

#### calculate agg aspect ratio, shape, and radius

In [ ]:
def shape(a,b,c):
    if (b-c) <= (a-b):
        return 'prolate'
    else:
        return 'oblate'

In [ ]:
#speed up shape function 
vfunc = np.vectorize(shape)
df_rand['shape'] = vfunc(df_rand['a'], df_rand['b'], df_rand['c'])
df_rand = df_rand.reset_index()
df_rand.loc[df_rand['shape'] == 'oblate', 'agg_r'] = np.power((np.power(df_rand['a'], 2) * df_rand['c']), (1./3.))
df_rand.loc[df_rand['shape'] == 'prolate', 'agg_r'] = np.power((np.power(df_rand['c'], 2) * df_rand['a']), (1./3.))
#df_rand = df_rand[df_rand.agg_r < 5000]
df_rand['agg_phi'] = df_rand.c/df_rand.a


In [ ]:
#speed up shape function 
vfunc = np.vectorize(shape)
df_flat['shape'] = vfunc(df_flat['a'], df_flat['b'], df_flat['c'])
df_flat = df_flat.reset_index()
df_flat.loc[df_flat['shape'] == 'oblate', 'agg_r'] = np.power((np.power(df_flat['a'], 2) * df_flat['c']), (1./3.))
df_flat.loc[df_flat['shape'] == 'prolate', 'agg_r'] = np.power((np.power(df_flat['c'], 2) * df_flat['a']), (1./3.))
df_flat['agg_phi'] = df_flat.c/df_flat.a


In [ ]:
df_flat['cplx'].describe().round(2)

In [ ]:
df_rand['agg_r'].describe().round(2)

In [ ]:
df_flat['agg_r'].describe().round(2)

In [ ]:
#how many monomers comprise the aggs when the agg radius is > 5mm?
#running this cell truncates the df and will lose the aggs > 5000
df_rand = df_rand[df_rand.agg_r > 5000]
df['ncrystals'].describe().round(2)

## Make bin plots

In [ ]:
agg_phi_bins = 20
agg_r_bins = 20

In [ ]:
def query_ncrystals(df_phi, r_bins):
    avg_ncrystals = []
    for r in range(len(r_bins)-1):
        df_r = df_phi[(df_phi.agg_r > r_bins[r]) & (df_phi.agg_r < r_bins[r+1])]
        avg_ncrystals.append(df_r.ncrystals.mean())
    return avg_ncrystals


In [ ]:
def avg_cplx(df_phi, r_bins):
    avg_cplx = []
    for r in range(len(r_bins)-1):
        df_r = df_phi[(df_phi.agg_r > r_bins[r]) & (df_phi.agg_r < r_bins[r+1])]
        avg_cplx.append(np.mean(df_r.cplx))
        
    return avg_cplx


In [ ]:
def oblate_prolate(df_phi, r_bins):
    oblates = []
    prolates = []
    for r in range(len(r_bins)-1):
        df_r = df_phi[(df_phi.agg_r > r_bins[r]) & (df_phi.agg_r < r_bins[r+1]) & (df_phi.ncrystals > 2)]
        oblate = df_r['shape'][df_r['shape'] == 'oblate'].count()
        prolate = df_r['shape'][df_r['shape'] == 'prolate'].count()
        oblates.append(oblate)
        prolates.append(prolate)
    return oblates, prolates

In [ ]:
def plate_columns_agg(df_phi, r_bins):
    agg_mono_phi = []
    for r in range(len(r_bins)-1):
        df_r = df_phi[(df_phi.agg_r > r_bins[r]) & (df_phi.agg_r < r_bins[r+1]) & (df_phi.ncrystals > 2)]
        agg_mono_plates = df_r['mono_phi'][df_r['mono_phi'] < 1.0].count()
        agg_mono_col = df_r['mono_phi'][df_r['mono_phi'] > 1.0].count()
        #agg_mono_phi.append(agg_mono_plates - agg_mono_col)
        agg_mono_phi.append(agg_mono_plates/(agg_mono_plates+agg_mono_col)*100)
    return agg_mono_phi


In [ ]:
def average_radius(df_phi, r_bins):
    avg_radius = []
    for r in range(len(r_bins)-1):
        df_r = df_phi[(df_phi.agg_r > r_bins[r]) & (df_phi.agg_r < r_bins[r+1])]
        avg_radius.append(np.mean(df_r.mono_r))
    return avg_radius

In [ ]:
df = df_flat
res, phi_bins = pd.qcut(df['agg_phi'], agg_phi_bins, retbins=True)
phi_bin_labs = []
avg_ncrystals=np.empty((len(phi_bins)-1,len(phi_bins)-1))
avg_cplxs=np.empty((len(phi_bins)-1,len(phi_bins)-1))
all_r_bins= np.empty((len(phi_bins),len(phi_bins)))
oblates= np.empty((len(phi_bins)-1,len(phi_bins)-1))
prolates= np.empty((len(phi_bins)-1,len(phi_bins)-1))
shape= np.empty((len(phi_bins)-1,len(phi_bins)-1))
agg_mono_phi = np.empty((len(phi_bins)-1,len(phi_bins)-1))
avg_radius = np.empty((len(phi_bins)-1,len(phi_bins)-1))

for i in range(agg_phi_bins):
    print('i = ', i)
    phi_bin_labs.append('[%.3f-%.3f)' %(phi_bins[i],phi_bins[i+1]))
    #return a df that only queries within an aspect ratio bin
    df_phi = df[(df['agg_phi'] > phi_bins[i]) & (df['agg_phi'] < phi_bins[i+1])]
    #now break that aspect ratio bin into 20 equal r bins
    res, r_bins = pd.qcut(df_phi.agg_r, agg_r_bins, retbins=True)
    all_r_bins[i,:] = r_bins
    
    #now use those r bins from the output of queried r and phi to find:
    avg_ncrystals[i,:] = query_ncrystals(df_phi, r_bins)
    avg_cplxs[i,:] = avg_cplx(df_phi, r_bins)
    oblates[i,:], prolates[i,:] = oblate_prolate(df_phi, r_bins)
    shape[i,:] = oblates[i,:] - prolates[i,:]
    agg_mono_phi[i,:] = plate_columns_agg(df_phi, r_bins)
    avg_radius[i,:] = average_radius(df_phi, r_bins)
    

In [ ]:
fig, ax = plt.subplots(figsize=(7,5))
cmap = plt.cm.jet
variable = avg_radius
norm = matplotlib.colors.LogNorm(vmin=1.13, vmax=672)

#norm = matplotlib.colors.Normalize(vmin=np.amin(shape), vmax=np.amax(shape))
for i in range(agg_phi_bins): 
    for r in range(agg_r_bins):
        #print(i,r)
        if r != 0:
            plt.bar([i]*20, all_r_bins[i,r+1], bottom= all_r_bins[i,r-1],  color=cmap(norm(variable[i,r])),edgecolor='k')
        else:
            plt.bar([i]*20, all_r_bins[i,r+1], color=cmap(norm(variable[i,r])), edgecolor='k')
                
plt.yscale('log')
#plt.ylim(1.,10000)

#plt.xticks([])
plt.xticks(np.arange(len(phi_bin_labs)), phi_bin_labs, rotation=90, ha="center")
#plt.ylabel("Aggregate Radius Bins")

#plt.xlabel("Aggregate Aspect Ratio ($\phi$) Bins") 
cb = plt.cm.ScalarMappable(norm=norm, cmap=cmap)
fig.colorbar(cb, orientation='vertical')
#cbar.ax.set_ylabel('Average Monomer Radius per Bin', orientation='horizontal')
plt.title('Quasi-Horizontal Orientation')
#plt.title('Random Orientation')
plt.tight_layout()
#plt.savefig('../plots/bins_flat_meanmono_r.png', format='png', dpi=300)


In [ ]:
fig, ax = plt.subplots(figsize=(7,5))
cmap = plt.cm.jet
variable = avg_cplxs
norm = matplotlib.colors.Normalize(vmin=0.4, vmax=1.0)
bounds = np.linspace(0.4, 1.0, 11)
norm = matplotlib.colors.BoundaryNorm(bounds, cmap.N)

for i in range(agg_phi_bins): 
    print('i= ', i)
    for r in range(agg_r_bins):
        if r != 0:
            plt.bar([i]*len(r_bins), all_r_bins[i,r+1], bottom= all_r_bins[i,r-1],  color=cmap(norm(variable[i,r]), 10),edgecolor='k')
        else:
            plt.bar([i]*len(r_bins), all_r_bins[i,r+1], color=cmap(norm(variable[i,r]), 10), edgecolor='k')

plt.yscale('log')
#plt.ylim(1.,10000)
plt.xticks(np.arange(len(phi_bin_labs)), phi_bin_labs, rotation=90, ha="center")
#plt.ylabel("Aggregate Radius Bins")
#plt.xlabel("Aggregate Aspect Ratio ($\phi$) Bins")  
cb = plt.cm.ScalarMappable(norm=norm, cmap=cmap)
fig.colorbar(cb, orientation='vertical')
plt.title('Quasi-Horizontal Orientation')
#plt.title('Random Orientation')
plt.tight_layout()
#plt.savefig('../plots/bins_flat_meancplx.png', format='png', dpi=300)


In [ ]:
fig, ax = plt.subplots(figsize=(7,5))
cmap = plt.cm.jet
variable = avg_cplxs
norm = matplotlib.colors.Normalize(vmin=0.4, vmax=1.0)
bounds = np.linspace(0.4, 1.0, 11)
norm = matplotlib.colors.BoundaryNorm(bounds, cmap.N)

for i in range(agg_phi_bins): 
    print('i= ', i)
    for r in range(agg_r_bins):
        if r != 0:
            plt.bar([i]*len(r_bins), all_r_bins[i,r+1], bottom= all_r_bins[i,r-1],  color=cmap(norm(variable[i,r]), 10),edgecolor='k')
        else:
            plt.bar([i]*len(r_bins), all_r_bins[i,r+1], color=cmap(norm(variable[i,r]), 10), edgecolor='k')

plt.yscale('log')
#plt.ylim(1.,10000)
plt.xticks(np.arange(len(phi_bin_labs)), phi_bin_labs, rotation=90, ha="center")
#plt.ylabel("Aggregate Radius Bins")
#plt.xlabel("Aggregate Aspect Ratio ($\phi$) Bins")  
cb = plt.cm.ScalarMappable(norm=norm, cmap=cmap)
fig.colorbar(cb, orientation='vertical')
plt.title('Quasi-Horizontal Orientation')
#plt.title('Random Orientation')
plt.tight_layout()
#plt.savefig('../plots/bins_flat_meancplx.png', format='png', dpi=300)


In [ ]:
fig, ax = plt.subplots(figsize=(10,8))
cmap = plt.cm.jet
variable = avg_ncrystals

norm = matplotlib.colors.Normalize(vmin=2, vmax=25)

for i in range(agg_phi_bins): 
    print('i= ', i)
    for r in range(agg_r_bins):

        if r != 0:
            if shape[i,r] == 'p': 
                plt.bar([i]*len(r_bins), all_r_bins[i,r+1], bottom= all_r_bins[i,r-1],  color=cmap(norm(variable[i,r])),edgecolor='k')
            else:
                 plt.bar([i]*len(r_bins), all_r_bins[i,r+1], bottom= all_r_bins[i,r-1],  color=cmap(norm(variable[i,r])),edgecolor='k')
        else:
            if shape[i,r] == 'p':
                plt.bar([i]*len(r_bins), all_r_bins[i,r+1], color=cmap(norm(variable[i,r])), edgecolor='k')
            else:
                plt.bar([i]*len(r_bins), all_r_bins[i,r+1], color=cmap(norm(variable[i,r])), edgecolor='k')
                
    
plt.yscale('log')
plt.xticks(np.arange(len(phi_bin_labs)), phi_bin_labs, rotation=90, ha="center",fontsize=16,family='serif')
plt.ylabel("Aggregate Radius Bins",fontsize=16,family='serif')
plt.xlabel("Aggregate Aspect Ratio ($\phi$) Bins",fontsize=16,family='serif')  
cb = plt.cm.ScalarMappable(norm=norm, cmap=cmap)
cbar = plt.colorbar(cb, format='%d')
cbar.ax.set_ylabel('Average # of Monomers per Bin', fontsize=16, family='serif')
plt.title('Quasi-Horizontal Orientation',fontsize=16, family='serif')
#plt.title('Random Orientation',fontsize=16, family='serif')
plt.tight_layout()


In [ ]:
fig, ax = plt.subplots(figsize=(10,8))
cmap = plt.cm.jet
variable = avg_ncrystals

norm = matplotlib.colors.Normalize(vmin=2, vmax=25)

for i in range(agg_phi_bins): 
    print('i= ', i)
    for r in range(agg_r_bins):

        if r != 0:
            if shape[i,r] == 'p': 
                plt.bar([i]*len(r_bins), all_r_bins[i,r], bottom= all_r_bins[i,r-1],  color=cmap(norm(variable[i,r])),edgecolor='k')
            else:
                 plt.bar([i]*len(r_bins), all_r_bins[i,r], bottom= all_r_bins[i,r-1],  color=cmap(norm(variable[i,r])),edgecolor='k')
        else:
            if shape[i,r] == 'p':
                plt.bar([i]*len(r_bins), all_r_bins[i,r], color=cmap(norm(variable[i,r])), edgecolor='k')
            else:
                plt.bar([i]*len(r_bins), all_r_bins[i,r], color=cmap(norm(variable[i,r])), edgecolor='k')
                
    
plt.yscale('log')
plt.xticks(np.arange(len(phi_bin_labs)), phi_bin_labs, rotation=90, ha="center",fontsize=16,family='serif')
plt.ylabel("Aggregate Radius Bins",fontsize=16,family='serif')
plt.xlabel("Aggregate Aspect Ratio ($\phi$) Bins",fontsize=16,family='serif')  
cb = plt.cm.ScalarMappable(norm=norm, cmap=cmap)
cbar = plt.colorbar(cb, format='%d')
cbar.ax.set_ylabel('Average # of Monomers per Bin', fontsize=16, family='serif')
plt.title('Quasi-Horizontal Orientation',fontsize=16, family='serif')
#plt.title('Random Orientation',fontsize=16, family='serif')
plt.tight_layout()


In [ ]:
fig, ax = plt.subplots(figsize=(7,5))
#cmap = plt.cm.RdBu_r
cmap=discrete_cmap(14,'RdBu_r')
variable=agg_mono_phi
# norm = matplotlib.colors.Normalize(vmin=-max(abs(np.amin(agg_mono_phi)),abs(np.amax(agg_mono_phi))), \
#                                    vmax=max(abs(np.amin(agg_mono_phi)),abs(np.amax(agg_mono_phi))))
norm1 = matplotlib.colors.Normalize(vmin=0, vmax=70)
norm = matplotlib.colors.DivergingNorm(vmin=0, vcenter=50., vmax=70)
for i in range(agg_phi_bins): 
    print('i= ', i)
    for r in range(agg_r_bins):  
        if r != 0:
            plt.bar(i, all_r_bins[i,r], bottom= all_r_bins[i,r-1], color=cmap(norm(variable[i,r])), edgecolor='k')
        else:
            plt.bar(i, all_r_bins[i,r], color=cmap(norm(variable[i,r])), edgecolor='k')

plt.yscale('log')
plt.ylim(1.,10000)
plt.xticks(np.arange(len(phi_bin_labs)), phi_bin_labs, rotation=90, ha="center")
# plt.ylabel("Aggregate Radius Bins")
# plt.xlabel("Aggregate Aspect Ratio ($\phi$) Bins",)  

cb = cm.ScalarMappable(cmap=cmap, norm=norm1)
cbar = plt.colorbar(cb, format='%d')
cbar.set_ticks(np.linspace(0,100,11))
          
#cbar.ax.set_ylabel('# of Oblates per Bin')
#plt.title('Quasi-Horizontal Orientation')
plt.title('Random Orientation');
#plt.tight_layout()
plt.tight_layout()
#plt.savefig('../plots/bins_flat_monophi.png', format='png',dpi=300)

In [ ]:
fig, ax = plt.subplots(figsize=(7,5))
#cmap = plt.cm.RdBu_r
cmap=discrete_cmap(14,'RdBu_r')
variable=agg_mono_phi
# norm = matplotlib.colors.Normalize(vmin=-max(abs(np.amin(agg_mono_phi)),abs(np.amax(agg_mono_phi))), \
#                                    vmax=max(abs(np.amin(agg_mono_phi)),abs(np.amax(agg_mono_phi))))
norm1 = matplotlib.colors.Normalize(vmin=0, vmax=70)
norm = matplotlib.colors.DivergingNorm(vmin=0, vcenter=50., vmax=70)
for i in range(agg_phi_bins): 
    print('i= ', i)
    for r in range(agg_r_bins):  
        if r != 0:
            plt.bar(i, all_r_bins[i,r], bottom= all_r_bins[i,r-1], color=cmap(norm(variable[i,r])), edgecolor='k')
        else:
            plt.bar(i, all_r_bins[i,r], color=cmap(norm(variable[i,r])), edgecolor='k')

plt.yscale('log')
plt.ylim(1.,10000)
plt.xticks(np.arange(len(phi_bin_labs)), phi_bin_labs, rotation=90, ha="center")
# plt.ylabel("Aggregate Radius Bins")
# plt.xlabel("Aggregate Aspect Ratio ($\phi$) Bins",)  

cb = cm.ScalarMappable(cmap=cmap, norm=norm1)
cbar = plt.colorbar(cb, format='%d')
cbar.set_ticks(np.linspace(0,100,11))
          
#cbar.ax.set_ylabel('# of Oblates per Bin')
plt.title('Quasi-Horizontal Orientation')
#plt.title('Random Orientation');
#plt.tight_layout()
plt.tight_layout()
#plt.savefig('../plots/bins_flat_monophi.png', format='png',dpi=300)